## BLE RSSI Dataset for Indoor localization

The dataset was created using the RSSI readings of an array of 13 ibeacons in the first floor of Waldo Library, Western Michigan University. Data was collected using iPhone 6S. The dataset contains two sub-datasets: a labeled dataset (1420 instances) and an unlabeled dataset (5191 instances). The recording was performed during the operational hours of the library. For the labeled dataset, the input data contains the location (label column), a timestamp, followed by RSSI readings of 13 iBeacons. RSSI measurements are negative values. Bigger RSSI values indicate closer proximity to a given iBeacon (e.g., RSSI of -65 represent a closer distance to a given iBeacon compared to RSSI of -85). For out-of-range iBeacons, the RSSI is indicated by -200. The locations related to RSSI readings are combined in one column consisting a letter for the column and a number for the row of the position. The following figure depicts the layout of the iBeacons as well as the arrange of locations.

# Configure docker credentials

Get your docker registry user and password encoded in base64 

echo -n USER:PASSWORD | base64 

Create a config.json file with your Docker registry url and the previous generated base64 string 

In [1]:
!echo -n USER:PASSWORD | base64

VVNFUjpQQVNTV09SRA==


In [2]:
%%writefile config.json
{
    "auths": {
        "https://index.docker.io/v1/": {
            "auth": "<<Provide previous generated base64 string>>"
        }
    }
}

Writing config.json


## Create requirements.txt

In [3]:
%%writefile requirements.txt
pandas
numpy
seldon-core
tornado>=6.0.3
kubeflow-fairing
tensorflow==1.14.0
cloudpickle==1.1.1

Writing requirements.txt


## Install the packages listed in requirements.txt using pip

In [4]:
!pip install -r requirements.txt --user 

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [1]:
import time
import re, os
import tensorflow as tf
import pandas as pd
import numpy as np
import logging
import sys
import importlib

/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

## Set up Kubeflow Fairing for training and predictions on On-premise
Import the fairing library and configure the onprem environment that your training or prediction job will run in.

In [2]:
from kubernetes import client as k8s_client
from kubernetes.client import rest as k8s_rest
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

from kubeflow import fairing   
from kubeflow.fairing import utils as fairing_utils
from kubeflow.fairing import TrainJob
from kubeflow.fairing.preprocessors.function import FunctionPreProcessor
from kubeflow.fairing.preprocessors import base as base_preprocessor
from kubeflow.fairing.builders.cluster.cluster import ClusterBuilder

from kubeflow.fairing.cloud.k8s import MinioUploader
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
from kubeflow.fairing import PredictionEndpoint
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.kubernetes.utils import mounting_pvc

namespace = fairing_utils.get_current_k8s_namespace()
print("Namespace : %s"%namespace)

Namespace : anonymous


## Get minio-service cluster IP to upload docker build context
#### Set DOCKER_REGISTRY
The DOCKER_REGISTRY variable is used to push the newly built image. 
Please change the variable to the registry for which you've configured credentials.

In [3]:
BASE_IMAGE="tensorflow/tensorflow:1.15.2-gpu"
DOCKER_REGISTRY = "samba07"

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
minio_service_endpoint = None

try:
    minio_service_endpoint = api_client.read_namespaced_service(name='minio-service', namespace='kubeflow').spec.cluster_ip
except ApiException as e:
    if e.status == 403:
        logging.warning(f"The service account doesn't have sufficient privileges "
                      f"to get the kubeflow minio-service. "
                      f"You will have to manually enter the minio cluster-ip. "
                      f"To make this function work ask someone with cluster "
                      f"priveleges to create an appropriate "
                      f"clusterrolebinding by running a command.\n"
                      f"kubectl create --namespace=kubeflow rolebinding "
                       "--clusterrole=kubeflow-view "
                       "--serviceaccount=${NAMESPACE}:default-editor "
                       "${NAMESPACE}-minio-view")
        logging.error("API access denied with reason: {e.reason}")

s3_endpoint = minio_service_endpoint
minio_endpoint = "http://"+s3_endpoint+":9000"
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"
print(minio_endpoint)


minio_uploader = MinioUploader(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

http://10.106.203.39:9000


### Create a config-map in the namespace you're using with the docker config

In [4]:
!kubectl create --namespace $namespace configmap docker-config --from-file=./config.json

configmap/docker-config created


## Declare Variables

In [5]:
BLE_RSSI = pd.read_csv('iBeacon_RSSI_Labeled.csv') #Labeled dataset

# Configure model options
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "/mnt/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "/mnt/")
TF_MODEL_TYPE = os.getenv("TF_MODEL_TYPE", "DNN")
TF_TRAIN_STEPS = int(os.getenv("TF_TRAIN_STEPS", 5000))
TF_BATCH_SIZE = int(os.getenv("TF_BATCH_SIZE", 50))
TF_LEARNING_RATE = float(os.getenv("TF_LEARNING_RATE", 0.001))


# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
INPUT_FEATURE="x"
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

## Definition of Serving Input Receiver Function

In [6]:
feature_columns =  make_feature_cols()
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns)

serving_input_receiver_fn2 = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[W 200420 12:35:08 deprecation:323] From /home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/feature_column/feature_column.py:806: NumericColumn._parse_example_spec (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
    Instructions for updating:
    The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


## Define training job

In [7]:
def train_evaluate_model(TF_MODEL_DIR,TF_LEARNING_RATE,TF_EXPORT_DIR,TF_TRAIN_STEPS):
    INPUT_FEATURE = 'x'
    NUM_CLASSES = 3

    b3001 = tf.feature_column.numeric_column(key='b3001',dtype=tf.float64)
    b3002 = tf.feature_column.numeric_column(key='b3002',dtype=tf.float64)
    b3003 = tf.feature_column.numeric_column(key='b3003',dtype=tf.float64)
    b3004 = tf.feature_column.numeric_column(key='b3004',dtype=tf.float64)
    b3005 = tf.feature_column.numeric_column(key='b3005',dtype=tf.float64)
    b3006 = tf.feature_column.numeric_column(key='b3006',dtype=tf.float64)
    b3007 = tf.feature_column.numeric_column(key='b3007',dtype=tf.float64)
    b3008 = tf.feature_column.numeric_column(key='b3008',dtype=tf.float64)
    b3009 = tf.feature_column.numeric_column(key='b3009',dtype=tf.float64)
    b3010 = tf.feature_column.numeric_column(key='b3010',dtype=tf.float64)
    b3011 = tf.feature_column.numeric_column(key='b3011',dtype=tf.float64)
    b3012 = tf.feature_column.numeric_column(key='b3012',dtype=tf.float64)
    b3013 = tf.feature_column.numeric_column(key='b3013',dtype=tf.float64)
    feature_columns = [b3001, b3002, b3003, b3004, b3005, b3006, b3007, b3008, b3009, b3010, b3011, b3012, b3013]

    df_full = pd.read_csv('iBeacon_RSSI_Labeled.csv') #Labeled dataset

    # Input Data Preprocessing 
    df_full = df_full.drop(['date'],axis = 1)
    df_full[FEATURES] = (df_full[FEATURES]-df_full[FEATURES].mean())/df_full[FEATURES].std()

    #Output Data Preprocessing
    dict = {'O02': 0,'P01': 1,'P02': 2,'R01': 3,'R02': 4,'S01': 5,'S02': 6,'T01': 7,'U02': 8,'U01': 9,'J03': 10,'K03': 11,'L03': 12,'M03': 13,'N03': 14,'O03': 15,'P03': 16,'Q03': 17,'R03': 18,'S03': 19,'T03': 20,'U03': 21,'U04': 22,'T04': 23,'S04': 24,'R04': 25,'Q04': 26,'P04': 27,'O04': 28,'N04': 29,'M04': 30,'L04': 31,'K04': 32,'J04': 33,'I04': 34,'I05': 35,'J05': 36,'K05': 37,'L05': 38,'M05': 39,'N05': 40,'O05': 41,'P05': 42,'Q05': 43,'R05': 44,'S05': 45,'T05': 46,'U05': 47,'S06': 48,'R06': 49,'Q06': 50,'P06': 51,'O06': 52,'N06': 53,'M06': 54,'L06': 55,'K06': 56,'J06': 57,'I06': 58,'F08': 59,'J02': 60,'J07': 61,'I07': 62,'I10': 63,'J10': 64,'D15': 65,'E15': 66,'G15': 67,'J15': 68,'L15': 69,'R15': 70,'T15': 71,'W15': 72,'I08': 73,'I03': 74,'J08': 75,'I01': 76,'I02': 77,'J01': 78,'K01': 79,'K02': 80,'L01': 81,'L02': 82,'M01': 83,'M02': 84,'N01': 85,'N02': 86,'O01': 87,'I09': 88,'D14': 89,'D13': 90,'K07': 91,'K08': 92,'N15': 93,'P15': 94,'I15': 95,'S15': 96,'U15': 97,'V15': 98,'S07': 99,'S08': 100,'L09': 101,'L08': 102,'Q02': 103,'Q01': 104}
    df_full['location'] = df_full['location'].map(dict)
    df_train=df_full.sample(frac=0.8,random_state=200)
    df_valid=df_full.drop(df_train.index)

    location_counts = BLE_RSSI.location.value_counts()
    x1 = np.asarray(df_train[FEATURES])
    y = np.asarray(df_train['location'])


    def formatFeatures(features):
        formattedFeatures = {}
        numColumns = features.shape[1]

        for i in range(0, numColumns):
            formattedFeatures["b"+str(3001+i)] = features[:, i]

        return formattedFeatures

    trainingFeatures = formatFeatures(x1)
    trainingCategories = y
    trainingFeatures

    # Train Input Function
    def train_input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y))
        dataset = dataset.batch(64).repeat(1000)
        return dataset

    # Test Input Function
 
    def eval_input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y))
        return dataset.batch(64).repeat(1000)

    distribution=tf.distribute.experimental.ParameterServerStrategy()
    print('Number of devices: {}'.format(distribution.num_replicas_in_sync))
    config = tf.estimator.RunConfig(train_distribute=distribution, model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=100)

    # Build 3 layer DNN classifier

    model = tf.estimator.DNNClassifier(hidden_units = [13,65,110],
                 feature_columns = feature_columns,
                 model_dir = TF_MODEL_DIR,
                 n_classes=105, config=config
               )

    export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn2)

    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                    max_steps=5000)

    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                                  steps=100,
                                  exporters=export_final,
                                  throttle_secs=1,
                                  start_delay_secs=1)

    # Train and Evaluate the model

    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

## Define Blerssi class to be used by Kubeflow fairing
## ( Must necessarily contain train() and predict() methods)

In [8]:
class BlerssiServe(object):
    
    def __init__(self):
        self.TF_DATA_DIR = "/tmp/data/"
        self.TF_MODEL_DIR = "/mnt/"
        self.TF_EXPORT_DIR = "/mnt/"
        self.TF_MODEL_TYPE = "DNN"
        self.TF_TRAIN_STEPS = 3000
        self.TF_BATCH_SIZE = 100
        self.TF_LEARNING_RATE = 0.01
        self.train_input = "iBeacon_RSSI_Labeled.csv"
        self.test_input = "iBeacon_RSSI_Unlabeled.csv"
        self.model = None
                 

    def train(self):
        model = train_evaluate_model(self.TF_MODEL_DIR,self.TF_LEARNING_RATE,self.TF_EXPORT_DIR,self.TF_TRAIN_STEPS)
        
    def predict(self,X,feature_names=None):
        
        global model_input1, model_input, predictor
        feature_col=["b3001", "b3002","b3003","b3004","b3005","b3006","b3007","b3008","b3009","b3010","b3011","b3012","b3013"]
        model_input1=tf.train.Example()
        for i in range(len(X)):
            model_input1.features.feature[feature_col[i]].float_list.value.append(X[i])
            
        path=os.path.join(os.getcwd(), "/mnt/")
        for dir in os.listdir(path):
            if re.match('[0-9]',dir):
                exported_path=os.path.join(path,dir)
                break
        
        # Open a Session to predict
        with tf.Session() as sess:
         tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
         model_input =model_input1
         
         predictor= tf.contrib.predictor.from_saved_model(exported_path,signature_def_key='predict')
         input_tensor=tf.get_default_graph().get_tensor_by_name("input_example_tensor:0")
            
         model_input=model_input.SerializeToString()
         output_dict= predictor({"examples":[model_input]})
        sess.close()
        
        response = output_dict.items()
        print(response)
        response1 = output_dict['class_ids']
        return response1

## Train an Blerssi model remotely on Kubeflow
Kubeflow Fairing packages the BlerssiServe class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on kubeflow.

In [10]:
with open('requirements.txt', 'r+') as f:
    packages=f.readlines()
    f.seek(0)
    for package in packages:
        if 'tensorflow' not in package:
            f.write(package)
    f.truncate()
BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), "KubernetesBackend")
train_job = TrainJob(BlerssiServe, input_files=["iBeacon_RSSI_Labeled.csv", "requirements.txt"],
                     pod_spec_mutators = [mounting_pvc(pvc_name="nfs1", pvc_mount_path="/mnt/")],
                     docker_registry=DOCKER_REGISTRY, base_docker_image=BASE_IMAGE, backend=BackendClass(build_context_source=minio_context_source))
train_job.submit()

[W 200420 12:38:31 tasks:62] Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
[I 200420 12:38:31 tasks:66] Building the docker image.
[I 200420 12:38:31 cluster:46] Building image using cluster builder.
[W 200420 12:38:31 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200420 12:38:31 base:107] Creating docker context: /tmp/fairing_context_gzfbz14c
[W 200420 12:38:31 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[W 200420 12:38:31 manager:296] Waiting for fairing-builder-cqb5v-nr7bl to start...
[W 200420 12:38:31 manager:296] Waiting for fairing-builder-cqb5v-nr7bl to start...
[W 200420 12:38:31 manager:296] Waiting for fairing-builder-cqb5v-nr7bl to start...
[W 200420 12:38:33 manager:296] Waiting for fairing-builder-cqb5v-nr7bl to start...
[I 200420 12:38:34 manager:302] Pod star

INFO[0001] Resolved base name tensorflow/tensorflow:1.15.2-gpu to tensorflow/tensorflow:1.15.2-gpu
INFO[0001] Resolved base name tensorflow/tensorflow:1.15.2-gpu to tensorflow/tensorflow:1.15.2-gpu
INFO[0001] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:da7b6c8a63bdafa77864e7e874664acfe939fdc140cb99940610c34b8c461cd0: no such file or directory
INFO[0002] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0003] Built cross stage deps: map[]
INFO[0003] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0003] Error while retrieving image from cache: getting file info: stat /cache/sha256:da7b6c8a63bdafa77864e7e874664acfe939fdc140cb99940610c34b8c461cd0: no such file or directory
INFO[0003] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0004] Unpacking rootfs as cmd COPY /app//requirements.txt /app/ requires it.
INFO[0053] Taking snapshot of full filesystem..

[W 200420 12:40:53 job:101] The job fairing-job-9gg25 launched.
[W 200420 12:40:53 manager:296] Waiting for fairing-job-9gg25-djgsl to start...
[W 200420 12:40:53 manager:296] Waiting for fairing-job-9gg25-djgsl to start...
[W 200420 12:44:30 manager:296] Waiting for fairing-job-9gg25-djgsl to start...
[W 200420 12:44:30 manager:296] Waiting for fairing-job-9gg25-djgsl to start...
[W 200420 12:44:33 manager:296] Waiting for fairing-job-9gg25-djgsl to start...
[I 200420 12:44:48 manager:302] Pod started running True


2020-04-20 12:44:51.127000: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-20 12:44:51.136672: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties:
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:19:00.0
2020-04-20 12:44:51.137735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 1 with properties:
name: Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:d8:00.0
2020-04-20 12:44:51.137941: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-20 12:44:51.139080: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-20 12:44:51.140139: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.

[W 200420 12:46:20 job:173] Cleaning up job fairing-job-9gg25...


'fairing-job-9gg25'

## Deploy the trained model to Kubeflow for predictions
Kubeflow Fairing packages the BlerssiServe class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [11]:
endpoint = PredictionEndpoint(BlerssiServe, input_files=["iBeacon_RSSI_Labeled.csv","requirements.txt"],
                              docker_registry=DOCKER_REGISTRY, base_docker_image=BASE_IMAGE,
                              pod_spec_mutators = [mounting_pvc(pvc_name="nfs1", pvc_mount_path="/mnt/")],
                              backend=BackendClass(build_context_source=minio_context_source))
endpoint.create()

[W 200420 12:46:34 tasks:62] Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
[I 200420 12:46:34 tasks:66] Building the docker image.
[I 200420 12:46:34 cluster:46] Building image using cluster builder.
[W 200420 12:46:34 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200420 12:46:34 base:107] Creating docker context: /tmp/fairing_context_qoedu95a
[W 200420 12:46:34 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[W 200420 12:46:35 manager:296] Waiting for fairing-builder-9thg2-hr2b8 to start...
[W 200420 12:46:35 manager:296] Waiting for fairing-builder-9thg2-hr2b8 to start...
[W 200420 12:46:35 manager:296] Waiting for fairing-builder-9thg2-hr2b8 to start...
[W 200420 12:46:37 manager:296] Waiting for fairing-builder-9thg2-hr2b8 to start...
[I 200420 12:46:37 manager:302] Pod star

INFO[0001] Resolved base name tensorflow/tensorflow:1.15.2-gpu to tensorflow/tensorflow:1.15.2-gpu
INFO[0001] Resolved base name tensorflow/tensorflow:1.15.2-gpu to tensorflow/tensorflow:1.15.2-gpu
INFO[0001] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:da7b6c8a63bdafa77864e7e874664acfe939fdc140cb99940610c34b8c461cd0: no such file or directory
INFO[0002] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0002] Built cross stage deps: map[]
INFO[0002] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0003] Error while retrieving image from cache: getting file info: stat /cache/sha256:da7b6c8a63bdafa77864e7e874664acfe939fdc140cb99940610c34b8c461cd0: no such file or directory
INFO[0003] Downloading base image tensorflow/tensorflow:1.15.2-gpu
INFO[0004] Unpacking rootfs as cmd COPY /app//requirements.txt /app/ requires it.
INFO[0054] Taking snapshot of full filesystem..

[I 200420 12:48:57 tasks:100] Deploying the endpoint.
[I 200420 12:48:57 serving:74] Cluster endpoint: http://fairing-service-4ccsm.anonymous.svc.cluster.local:5000/predict
[W 200420 12:48:57 tasks:106] Prediction endpoint: http://fairing-service-4ccsm.anonymous.svc.cluster.local:5000/predict


## Wait for  prediction pod ready state

In [12]:
!kubectl get deploy -l fairing-deployer=serving -n anonymous

NAME                     READY   UP-TO-DATE   AVAILABLE   AGE
fairing-deployer-k74rr   1/1     1            1           5m50s


##  Get prediction endpoint

In [13]:
endpoint.url

'http://fairing-service-4ccsm.anonymous.svc.cluster.local:5000/predict'

## Call the prediction endpoint
Use the endpoint from previous cell

In [14]:
! curl -v http://fairing-service-4ccsm.anonymous.svc.cluster.local:5000/predict -H "Content-Type: application/x-www-form-urlencoded" -d 'json={"data":{"ndarray":[-0.458086, -0.6244, 2.354243, -0.404581, 1.421444, 1.767642, 2.637829, -0.603085, 0.382779, -0.378999, -0.341798, -0.303249, -0.327776]}}'

*   Trying 10.109.36.194...
* TCP_NODELAY set
* Connected to fairing-service-4ccsm.anonymous.svc.cluster.local (10.109.36.194) port 5000 (#0)
> POST /predict HTTP/1.1
> Host: fairing-service-4ccsm.anonymous.svc.cluster.local:5000
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Type: application/x-www-form-urlencoded
> Content-Length: 162
> 
* upload completely sent off: 162 out of 162 bytes
* HTTP 1.0, assume close after body
< HTTP/1.0 200 OK
< Content-Type: application/json
< Content-Length: 54
< Access-Control-Allow-Origin: *
< Server: Werkzeug/0.16.1 Python/3.6.9
< Date: Mon, 20 Apr 2020 12:55:05 GMT
< 
{"data":{"names":["t:0"],"ndarray":[[13]]},"meta":{}}
* Closing connection 0


## Clean up the prediction endpoint
Delete the prediction endpoint created by this notebook.

In [15]:
endpoint.delete()

[I 200420 12:56:45 tasks:130] Deleting the endpoint. 
[I 200420 12:56:45 serving:124] Deleted service: anonymous/fairing-service-4ccsm
[I 200420 12:56:45 serving:136] Deleted deployment: anonymous/fairing-deployer-k74rr


## Delete config.json and requirements.txt

In [16]:
rm -rf config.json requirements.txt